In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import os
import numpy as np

from VelocytoAnalysis import RNA_ODE

# Load Data

In [2]:
dir_data = 'data'

# Counts
counts = np.loadtxt(os.path.join(dir_data, 'counts.csv'), delimiter=",")
# Velocity
velocity = np.loadtxt(os.path.join(dir_data, 'velocity.csv'), delimiter=",")
# Cellular States. Run Clustering Algorithm is not known
celltype = np.loadtxt(os.path.join(dir_data, 'milestones.csv'), delimiter=",")

# Build RNA_ODE class

In [3]:
rna_ode = RNA_ODE(counts, velocity, celltype)

# Trajectory Inference

In [4]:
# Fit model
model = rna_ode.build_model()

# ODE Simulation
path= rna_ode.ode_simulation()

# Compute Lineages
lineages = rna_ode.compute_lineages()

Fitted model | Training R-Square: 0.8946; Test R-Square: 0.7386
ODE Simulation Done.
Root is 4.0


Evaluate lineages if true lineage is known

In [5]:
# True lineages - cellular states which have direct transitions
lineages_true = np.loadtxt(os.path.join(dir_data, 'lineages.csv'), delimiter=",")
correctness = rna_ode.evaluate_lineage_correctness(lineages_true)
print('Correctness of Lineages: %.3f' % correctness)

Correctness of Lineages: 0.667


# Gene Regulatory Network

In [6]:
# pair-wise regulatory links score matrix - GENIE3: expression to expression
grn_scores_genie3 = rna_ode.compute_grn_scores(method='GENIE3')
# pair-wise regulatory links score matrix - RNA_ODE: expression to velocity
grn_scores_rnaode = rna_ode.compute_grn_scores(method='RNA_ODE')

Tree method: RF
K: sqrt
Number of trees: 1000
Elapsed time: 11.95 seconds

Tree method: RF
K: sqrt
Number of trees: 1000
Elapsed time: 11.88 seconds



Evaluate GRN if true GRN is known

In [7]:
# True GRN - pairs of true regulatory links
grn_true = np.loadtxt(os.path.join(dir_data, 'grn.csv'), delimiter=",")

auroc_genie3 = rna_ode.evaluate_grn_auroc(grn_true, grn_scores_genie3)
auroc_rnaode = rna_ode.evaluate_grn_auroc(grn_true, grn_scores_rnaode)
print('AUROC score for GRN | GENIE3: %.4f; RNA_ODE: %.4f' % (auroc_genie3, auroc_rnaode)) 

AUROC score for GRN | GENIE3: 0.6162; RNA_ODE: 0.6303
